# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
%load_ext dotenv
%dotenv

In [2]:
import dask.dataframe as dd

c:\Users\Catren-PC\miniconda3\envs\dsi_cat\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [6]:
import os
from glob import glob

# load env variable PRICE_DATA 
PRICE_DATA= os.getenv("PRICE_DATA")

#find all parquet files in the directory
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [7]:
#use Dask to read files into dataframes and index on Ticker
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")
dd_px

,Date,Adj Close,Close,High,Low,Open,Volume,Year
npartitions=64,,,,,,,,
AAPL,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32
ACN,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
ZBRA,...,...,...,...,...,...,...,...
ZBRA,...,...,...,...,...,...,...,...


In [11]:
#Add lags for Close and Adj_Close
dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1), 
        Adjclose_lag_1 = x['Adj Close'].shift(1)
        )
)

#Check a few rows
dd_shift.head(5)

C:\Users\Catren-PC\AppData\Local\Temp\ipykernel_8860\2547955453.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(


,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adjclose_lag_1
Ticker,,,,,,,,,,
AAPL,2000-01-03 00:00:00+00:00,0.843077,0.999442,1.004464,0.907924,0.936384,535796800.0,2000,NaN,NaN
AAPL,2000-01-04 00:00:00+00:00,0.771997,0.915179,0.987723,0.903460,0.966518,512377600.0,2000,0.999442,0.843077
AAPL,2000-01-05 00:00:00+00:00,0.783293,0.928571,0.987165,0.919643,0.926339,778321600.0,2000,0.915179,0.771997
AAPL,2000-01-06 00:00:00+00:00,0.715509,0.848214,0.955357,0.848214,0.947545,767972800.0,2000,0.928571,0.783293
AAPL,2000-01-07 00:00:00+00:00,0.749402,0.888393,0.901786,0.852679,0.861607,460734400.0,2000,0.848214,0.715509


In [16]:
#Add returns and range, put into dd_feat
dd_feat = dd_shift.assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1,
    hi_lo_range = lambda x: x['High'] - x['Low']
)

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [68]:
#Convert to pandas df
dd_feat_pd= dd_feat.compute()

#check to confirm pandas df
type(dd_feat_pd)

pandas.core.frame.DataFrame

In [71]:
#Sort the pandas df
dd_feat_pd = (dd_feat_pd
           .reset_index()
           .sort_values(['Ticker', 'Date']))

dd_feat_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403456 entries, 0 to 403455
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   Ticker          403456 non-null  object             
 1   Date            403456 non-null  datetime64[ns, UTC]
 2   Adj Close       357824 non-null  float64            
 3   Close           357824 non-null  float64            
 4   High            357824 non-null  float64            
 5   Low             357824 non-null  float64            
 6   Open            357824 non-null  float64            
 7   Volume          357824 non-null  float64            
 8   Year            403456 non-null  int32              
 9   Close_lag_1     357760 non-null  float64            
 10  Adjclose_lag_1  357760 non-null  float64            
 11  returns         357760 non-null  float64            
 12  hi_lo_range     357824 non-null  float64            
dtypes: datetime64[

In [ ]:
# Add column for rolling avg
# Source for using groupby and getting values: https://stackoverflow.com/questions/60703071/how-to-include-moving-average-with-pandas-based-on-values-on-other-columns

dd_feat_pd['returns_10day_avg']= dd_feat_pd.groupby('Ticker')['returns'].rolling(10).mean().values

In [84]:
#Check results for a few tickers e.g. ZBRA, ADBE
dd_feat_pd[dd_feat_pd['Ticker'] == 'ADBE'].head(30)

,Ticker,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adjclose_lag_1,returns,hi_lo_range,returns_10day_avg
12608,ADBE,2000-01-03 00:00:00+00:00,16.274672,16.390625,16.875000,16.062500,16.812500,7384400.0,2000,NaN,NaN,NaN,0.812500,NaN
12609,ADBE,2000-01-04 00:00:00+00:00,14.909399,15.015625,16.453125,14.984375,15.750000,7813200.0,2000,16.390625,16.274672,-0.083889,1.468750,NaN
12610,ADBE,2000-01-05 00:00:00+00:00,15.204175,15.312500,15.687500,14.562500,14.562500,14927200.0,2000,15.015625,14.909399,0.019771,1.125000,NaN
12611,ADBE,2000-01-06 00:00:00+00:00,15.328290,15.437500,15.656250,15.156250,15.359375,10221200.0,2000,15.312500,15.204175,0.008163,0.500000,NaN
12612,ADBE,2000-01-07 00:00:00+00:00,16.072985,16.187500,16.187500,15.218750,15.390625,8253200.0,2000,15.437500,15.328290,0.048583,0.968750,NaN
12613,ADBE,2000-01-10 00:00:00+00:00,16.693565,16.812500,16.812500,15.906250,16.343750,10987200.0,2000,16.187500,16.072985,0.038610,0.906250,NaN
12614,ADBE,2000-01-11 00:00:00+00:00,15.545493,15.656250,16.812500,15.593750,16.796875,9616000.0,2000,16.812500,16.693565,-0.068773,1.218750,NaN
12615,ADBE,2000-01-12 00:00:00+00:00,15.467920,15.578125,15.843750,15.437500,15.687500,8051200.0,2000,15.656250,15.545493,-0.004990,0.406250,NaN
12616,ADBE,2000-01-13 00:00:00+00:00,16.290182,16.406250,16.500000,15.687500,15.703125,5527200.0,2000,15.578125,15.467920,0.053159,0.812500,NaN
12617,ADBE,2000-01-14 00:00:00+00:00,16.460842,16.578125,16.843750,16.406250,16.656250,5136800.0,2000,16.406250,16.290182,0.010476,0.437500,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

It was not necessary to convert to pandas df to calculate the moving average because there's also a Dask rolling function that takes Dask dataframes. It might have been better to do it in Dask if we didn't need the result immediately and wanted to save computing power through lazy execution (without running the compute function). 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [X] Created a branch with the correct naming convention.
- [X] Ensured that the repository is public.
- [X] Reviewed the PR description guidelines and adhered to them.
- [X] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.